## Creando el dataset

In [1]:
from prepare_data import run_scraper

run_scraper(
    out_csv="horoscopos.csv",
    max_pages=30,
    max_workers=8
)

[1/3] Indexando artículos desde el API…
   → 711 artículos encontrados
[2/3] Scrapeando artículos en paralelo…
[3/3] Listo. Artículos OK: 711 | con error: 0
CSV: horoscopos.csv


In [3]:
import pandas as pd
data = pd.read_csv("horoscopos.csv")
print(data.head())

        fecha    signo                                         prediccion
0  2025-09-29    ARIES  Hoy los celos te pondrán en una situación incó...
1  2025-09-29    TAURO  No dudes de las intenciones de esa persona esp...
2  2025-09-29  GÉMINIS  En medio de rumores que harían dudar a cualqui...
3  2025-09-29   CÁNCER  Tu amor se pondrá a prueba, descubrirás una me...
4  2025-09-29      LEO  Tienes muchas dudas con respecto a tu vida sen...


In [4]:
print("Hay ", len(data), "predicciones en el dataset.")

Hay  3654 predicciones en el dataset.


## Creando el modelo de lenguaje

In [5]:
import csv
import random
import re
from collections import defaultdict
from typing import Dict, List, Tuple

import nltk
from nltk import trigrams
from nltk.tokenize import sent_tokenize, word_tokenize

try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt")

try:
    nltk.data.find("tokenizers/punkt_tab")
except LookupError:
    try:
        nltk.download("punkt_tab")
    except Exception:
        pass

In [7]:
def tokenize_es(texto: str) -> List[List[str]]:
    sentences = sent_tokenize(texto, language="spanish")
    return [word_tokenize(o, language="spanish") for o in sentences]

In [10]:
def load_corpus(csv_path: str, by_sign: bool = False) -> Dict[str, List[List[str]]]:
    buckets: Dict[str, List[List[str]]] = defaultdict(list)
    with open(csv_path, encoding="utf-8", newline="") as f:
        r = csv.DictReader(f)
        for row in r:
            sign = (row.get("signo") or "").strip().upper()
            pred = row.get("prediccion")
            if not pred:
                continue
            sents_toks = tokenize_es(pred)
            if by_sign and sign:
                for s in sents_toks:
                    if s:
                        buckets[sign].append(s)
            else:
                for s in sents_toks:
                    if s:
                        buckets["_GLOBAL"].append(s)
    return buckets

In [11]:
global_corpus = load_corpus("horoscopos.csv", by_sign=False)

In [13]:
def train_trigrams(tokenized_sentences: List[List[str]]):
    """
      model[(w1,w2)][w3] = prob
    """
    model = defaultdict(lambda: defaultdict(float))
    for sent in tokenized_sentences:
        for w1, w2, w3 in trigrams(sent, pad_left=True, pad_right=True):
            model[(w1, w2)][w3] += 1.0

    for w1w2 in model:
        total = sum(model[w1w2].values())
        if total > 0:
            for w3 in model[w1w2]:
                model[w1w2][w3] /= total
    return model

global_model = train_trigrams(global_corpus["_GLOBAL"])
len(global_model)

39149

In [17]:
from typing import Optional

def sample_next(model: dict, w1: str, w2: str) -> Optional[str]:
    dist = model.get((w1, w2), {})
    if not dist:
        for ctx in [(None, w2), (w1, None), (None, None)]:
            dist = model.get(ctx, {})
            if dist:
                break
        if not dist:
            return None

    r = random.random()
    acc = 0.0
    for w3, p in dist.items():
        acc += p
        if acc >= r:
            return w3
    return next(iter(dist.keys()))

def generate_sentence(model: dict, seeds: Tuple[str, str] = (None, None), max_len: int = 30) -> str:
    text: List[str] = [seeds[0], seeds[1]]
    sentence_finished = False

    while not sentence_finished and len(text) < max_len + 2:
        w3 = sample_next(model, text[-2], text[-1])
        text.append(w3)
        if text[-2:] == [None, None] or w3 is None:
            sentence_finished = True

    sentence = " ".join([t for t in text if t])
    sentence = sentence.strip()
    if sentence and sentence[-1] not in ".!?":
        sentence += "."
    if sentence:
        sentence = sentence[0].upper() + sentence[1:]
    return sentence

def generate_paragraph(model: dict, n_sentences: int = 3, seeds: Tuple[str, str] = (None, None)) -> str:
    return " ".join(generate_sentence(model, seeds=seeds) for _ in range(n_sentences))

In [18]:
print("== Muestra (GLOBAL) ==")
print(generate_paragraph(global_model, n_sentences=3, seeds=(None, None)))

== Muestra (GLOBAL) ==
Tu vida amorosa se ha convertido en motivo de trabajo ; te organizarás para tener todo bajo control y no dejarás pasar una más . Tu optimismo contagiará a tus problemas sentimentales debes empezar por tener una relación más consolidada volverá a perturbarte . Hoy no pararás hasta solucionarlos .


## Persistencia del modelo

In [24]:
import json, gzip
from collections import defaultdict

def save_trigram_json(model: dict, path: str, metadata=None) -> None:
    rows = []
    for (w1, w2), dist in model.items():
        rows.append({
            "w1": w1, "w2": w2,
            "next": [{"w3": w3, "p": float(p)} for w3, p in dist.items()]
        })
    payload = {"format": "trigram-v1", "metadata": metadata or {}, "rows": rows}
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False)

save_trigram_json(global_model, "trigram_global.json", metadata={"scope": "GLOBAL"})

# Un modelo por Signo Zodiacal

In [25]:
SIGNS = [
    "ARIES","TAURO","GÉMINIS","CÁNCER","LEO","VIRGO",
    "LIBRA","ESCORPIO","SAGITARIO","CAPRICORNIO","ACUARIO","PISCIS"
]

def load_corpus_by_sign(csv_path: str) -> Dict[str, List[List[str]]]:
    return load_corpus(csv_path, by_sign=True)

sign_corpus = load_corpus_by_sign("horoscopos.csv")

models_by_sign: Dict[str, dict] = {}
for s in SIGNS:
    if s in sign_corpus and sign_corpus[s]:
        models_by_sign[s] = train_trigrams(sign_corpus[s])

for s in SIGNS:
    if s in models_by_sign:
        print(f"\n== {s} ==")
        print(generate_paragraph(models_by_sign[s], n_sentences=3, seeds=(None, None)))
    else:
        print(f"\n== {s} == (sin suficientes datos)")


== ARIES ==
Hoy la suerte es el diálogo . Estarás muy enamoradizo , pero debes tener paciencia , los objetivos ya están planteados , pero no tendrá la paciencia de siempre , hoy lo sentirás muy vulnerable , lo. Tu sentido de la llegada de un compañero te mortificara , no te desanimes porque aún no encuentras algo que te servirá para reflexionar .

== TAURO ==
Estarás de muy buen ánimo , irradiarás positivismo lo que lograrás equilibrar tu economía , es el momento ideal para concretarlo , contarás con apoyo . Es una etapa de renovación . Número de suerte 12 .

== GÉMINIS ==
Sentirás una gran atracción por esa persona que amas , lograrás lo que digas , tendrás que organizarte muy bien aspectada , se perseverante y alcanzarás el éxito será más. Número de suerte , 19 . No te dejes llevar por lo que imaginas , no te conviene trabajar solo tu nuevo proyecto .

== CÁNCER ==
La estabilidad que lograrás es ahondar tus problemas . Con mucho tacto para que la entrega de tu romance y decidirás to

In [27]:
for s, m in models_by_sign.items():
    save_trigram_json(m, f"trigram_{s}.json", metadata={"scope": s})
    